# Scan Data Last BVA and LVA

refresh every week BVA25_session_export.csv. Rest from scan vistors BVA and LVE previous year dont change

In [ ]:
import pandas as pd
import json
from pandas import json_normalize
import warnings
from typing import List
from pandas.errors import SettingWithCopyWarning
import logging

warnings.simplefilter(action="ignore", category=(SettingWithCopyWarning))

# Scan Data

In [ ]:
session_this_path = "data/bva/BVA25_session_export.csv"
session_past_path_bva = "data/bva/BVA24_session_export.csv"
session_past_path_lva = "data/bva/LVA24_session_export.csv"
seminars_scan_reference_past_path_bva = "data/bva/bva2024 seminar scans reference.csv"
seminars_scans_past_path_bva = "data/bva/bva2024 seminar scans.csv"
seminars_scan_reference_past_path_lva = "data/bva/lvs2024 seminar scans reference.csv"
seminars_scans_past_path_lva = "data/bva/lvs2024 seminar scans.csv"

In [ ]:
session_this = pd.read_csv(session_this_path)
session_past_bva = pd.read_csv(session_past_path_bva)
session_past_lva = pd.read_csv(session_past_path_lva)
seminars_scan_reference_past_bva = pd.read_csv(seminars_scan_reference_past_path_bva)
seminars_scans_past_bva = pd.read_csv(seminars_scans_past_path_bva)

seminars_scan_reference_past_lva = pd.read_csv(seminars_scan_reference_past_path_lva)
seminars_scans_past_lva = pd.read_csv(seminars_scans_past_path_lva)

In [ ]:
seminars_scan_reference_past_bva.columns

In [ ]:
seminars_scans_past_enhanced_bva = pd.merge(
    seminars_scans_past_bva,
    seminars_scan_reference_past_bva[["Short Name", "Seminar Name"]],
    on=["Short Name"],
    how="left",
)
len(seminars_scans_past_enhanced_bva), len(seminars_scans_past_bva)

In [ ]:
seminars_scans_past_enhanced_bva.isnull().sum()

In [ ]:
seminars_scans_past_enhanced_bva.head()

In [ ]:
seminars_scans_past_enhanced_lva = pd.merge(
    seminars_scans_past_lva,
    seminars_scan_reference_past_lva[["Short Name", "Seminar Name"]],
    on=["Short Name"],
    how="left",
)
len(seminars_scans_past_enhanced_lva), len(seminars_scans_past_lva)

In [ ]:
import string


def clean_text(text):
    # Remove punctuation and spaces, and convert to lowercase
    return "".join(char for char in text if char.isalnum()).lower()

In [ ]:
seminars_scans_past_enhanced_bva["key_text"] = seminars_scans_past_enhanced_bva[
    "Seminar Name"
].apply(clean_text)
seminars_scans_past_enhanced_lva["key_text"] = seminars_scans_past_enhanced_lva[
    "Seminar Name"
].apply(clean_text)

In [ ]:
session_past_bva["key_text"] = session_past_bva["title"].apply(clean_text)
session_past_lva["key_text"] = session_past_lva["title"].apply(clean_text)

# BVA 2024

In [ ]:
list1 = set(list(seminars_scans_past_enhanced_bva["key_text"].unique()))
list2 = set(list(session_past_bva["key_text"].unique()))

In [ ]:
len(list1.intersection(list2)), len(list2), len(list1)

In [ ]:
reg_data_last_bva = pd.read_csv("data/bva/output/df_reg_demo_last_bva.csv")

In [ ]:
list3 = set(list(reg_data_last_bva["BadgeId"].unique()))
list4 = set(list(seminars_scans_past_enhanced_bva["Badge Id"].unique()))

In [ ]:
len(list4.intersection(list3)), len(list3), len(list4)

In [ ]:
seminars_scans_past_enhanced_bva.to_csv(
    "data/bva/output/scan_bva_past.csv", index=False
)
len(seminars_scans_past_enhanced_bva)

# LVA 2024

In [ ]:
list5 = set(list(seminars_scans_past_enhanced_lva["key_text"].unique()))
list6 = set(list(session_past_lva["key_text"].unique()))

In [ ]:
len(list5.intersection(list6)), len(list5), len(list6)

In [ ]:
reg_data_last_lva = pd.read_csv("data/bva/output/df_reg_demo_last_lva.csv")

In [ ]:
list7 = set(list(reg_data_last_lva["BadgeId"].unique()))
list8 = set(list(seminars_scans_past_enhanced_lva["Badge Id"].unique()))

In [ ]:
len(list8.intersection(list7)), len(list7), len(list8)

In [ ]:
seminars_scans_past_enhanced_lva.to_csv(
    "data/bva/output/scan_lva_past.csv", index=False
)
len(seminars_scans_past_enhanced_lva)

In [ ]:
df_reg_24_wdemo_data_bva = pd.read_csv(
    "data/bva/output/Registration_data_with_demographicdata_bva_last.csv"
)
df_reg_24_wdemo_data_lva = pd.read_csv(
    "data/bva/output/Registration_data_with_demographicdata_lva_last.csv"
)

In [ ]:
len(df_reg_24_wdemo_data_bva), len(df_reg_24_wdemo_data_lva)

In [ ]:
seminars_scans_past_enhanced_bva.columns

In [ ]:
import pandas as pd


def add_demographics_to_seminars(
    df_reg_24_wdemo_data_bva, seminars_scans_past_enhanced_bva
):
    """
    Add demographic data to seminar scans dataframe for matching badge IDs.

    Parameters:
    -----------
    df_reg_24_wdemo_data_bva : pandas.DataFrame
        Dataframe containing registration and demographic data
    seminars_scans_past_enhanced_bva : pandas.DataFrame
        Dataframe containing seminar attendance data

    Returns:
    --------
    pandas.DataFrame
        Enhanced seminar dataframe with demographic data added
    """
    # Create copies to avoid modifying original dataframes
    registration_df = df_reg_24_wdemo_data_bva.copy()
    seminars_df = seminars_scans_past_enhanced_bva.copy()

    # Rename 'BadgeId' column in registration dataframe to match 'Badge Id' in seminars dataframe
    # This makes the merge operation simpler
    registration_df = registration_df.rename(columns={"BadgeId": "Badge Id"})

    # Select only the columns we want to merge from the registration dataframe
    # You can adjust this list to include any demographic columns you need
    demographic_columns = [
        "Badge Id",
        "Title",
        "Forename",
        "Surname",
        "Email",
        "Company",
        "JobTitle",
        "Country",
        "BadgeType",
        "Email_domain",
    ]

    # Create a streamlined registration dataframe with only the columns we need
    reg_slim = registration_df[demographic_columns]

    # Merge the seminars dataframe with the registration dataframe
    # Using left join to keep all seminar records, even if there's no matching badge ID
    enhanced_seminars = pd.merge(seminars_df, reg_slim, on="Badge Id", how="left")

    # Print some merge statistics
    total_seminars = len(seminars_df)
    matched_seminars = enhanced_seminars["Email"].notna().sum()
    print(f"Total seminar records: {total_seminars}")
    print(
        f"Records with matching badge IDs: {matched_seminars} ({matched_seminars/total_seminars*100:.2f}%)"
    )

    return enhanced_seminars


# Example usage:
# enhanced_seminars_df = add_demographics_to_seminars(df_reg_24_wdemo_data_bva, seminars_scans_past_enhanced_bva)

In [ ]:
enhanced_seminars_df_bva = add_demographics_to_seminars(
    df_reg_24_wdemo_data_bva, seminars_scans_past_enhanced_bva
)

In [ ]:
enhanced_seminars_df_lve = add_demographics_to_seminars(
    df_reg_24_wdemo_data_lva, seminars_scans_past_enhanced_lva
)

In [ ]:
enhanced_seminars_df_bva.to_csv(
    "data/bva/output/sessions_visited_last_bva.csv", index=False
)
enhanced_seminars_df_lve.to_csv(
    "data/bva/output/sessions_visited_last_lve.csv", index=False
)